In [16]:
import pickle
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp

In [28]:
hand_sign_dict = {
    1: "A",
    2: "B", 
    3: "C",
    4: "D",
    5: "E",
    6: "F",
    7: "G",
    8: "H",
    9: "I",
    10: "J",
    11: "K",
    12: "L",
    13: "M",
    14: "N",
    15: "O",
    16: "P",
    17: "Q",
    18: "R",
    19: "S",
    20: "T",
    21: "U",
    22: "V",
    23: "W",
    24: "X",
    25: "Y",
    26: "Z"
}

DF_COLUMNS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41']

In [33]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
flag_df_not_empty = False
model = pickle.load(open("Best_Model_1.pkl", 'rb'))
try:
    cap = cv2.VideoCapture(0)
    with mp_hands.Hands(min_detection_confidence = 0.5, max_num_hands = 1, static_image_mode = False) as hands:
        while True:
            success, image = cap.read()
            image = cv2.flip(image, 1)
            if not success:
                print("Error reading frame from camera")
                break
    
            results = hands.process(image)
            coords = list()
            coords_list = list()
            if results.multi_hand_landmarks:
                for value in results.multi_hand_landmarks[0].landmark:
                    coords.append(value.x)
                    coords.append(value.y)
                coords_list.append(coords)
            temp = pd.DataFrame(coords_list, columns = DF_COLUMNS)
            if not temp.empty:
                pred = model.predict(temp)
                flag_df_not_empty = True
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                             mp_drawing.DrawingSpec(color=(0,255,0), thickness = 2, circle_radius = 4))
            if flag_df_not_empty:
                cv2.putText(image, f'Hand Sign: {hand_sign_dict[pred[0]]}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
                            cv2.LINE_AA)
                flag_df_not_empty = False
            cv2.imshow(f"Predicting", image)
            # Check for the 'q' key to exit
            if cv2.waitKey(1) == ord("q"):
                cap.release()
                cv2.destroyAllWindows()
                cv2.waitKey(1)
                break
                
# Incase anything goes wrong, close camera capture
finally:
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

I0000 00:00:1715900459.741679       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro


ValueError: Shape of passed values is (42, 1), indices imply (42, 42)